In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
from backtester.analysis import *
from backtester.swarms.swarm import Swarm
from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage


import pandas as pd
import numpy as np
import scipy
import pprint

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

In [ ]:
storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)

# Stage 1: Campaign composition

### List of available products in Mongo

In [ ]:
swm_info = storage.swarms_info()
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(swm_info)

## Select product

In [ ]:
product_name = 'ES'

## Getting swarm using filter

In [ ]:
instruments_filter = [product_name]  # Select ALL
#instruments_filter = ['ES', 'CL']

exo_filter = ['*'] # Select ALL
#exo_filter = ['BullishCollar']

direction_filter = [0, -1, 1] # Select ALL
#direction_filter = [1]

alpha_filter = ['-custom', 'SMART', '_EXO']  # Select ALL
#alpha_filter = ['*']

swmdf, swm_data = storage.swarms_list(instruments_filter, direction_filter, alpha_filter, exo_filter)

## Swarms loaded

In [ ]:
[print(s) for s in sorted(swmdf.columns)];

### Swarms equities

#### All-on-one

In [ ]:
swmdf.plot(legend=False);

#### Individual plots

In [ ]:
figsize(15, 8)
for s in swm_data:    
    series = s['swarm_series']
    
    if 'delta' in series:
        # Two subplots, the axes array is 1-d
        f, axarr = plt.subplots(2, gridspec_kw = {'height_ratios':[3, 1, 1]})
        
        series['equity'].plot(ax=axarr[0])
        axarr[0].set_title('Alpha name: {0}'.format(s['swarm_name']))
        series['delta'].plot(ax=axarr[1])
        axarr[1].set_title('Delta')
    else:
        series['equity'].plot(title='Alpha name: {0}'.format(s['swarm_name']))
        
        
    plt.show()

## Individual swarm equity

In [ ]:
_swarm_name = "CL_PutSpread_Short_SwingPoint_bearish-custom"

swmdf[_swarm_name].plot();
title(_swarm_name);

### Available products EXOs

#### Alpha names help

* 'CL_CallSpread_Long_EXO' - vanilla long EXO
* 'CL_ContFut_Long_EXO' - continuous futures EXO
* 'CL_SmartEXO_Ichi_Put_Spread_150Delta_Bi_Long_EXO' - smart EXO (as is, without alphas)

In [ ]:
print('==== Alphas EXO list')
[print("'{0}': 1.0,".format(s)) for s in sorted(swmdf.columns)];

# Stage2: Campaign building

Copy/paste EXO or swarm names to the 'campaign_exposure' dictionary from list above

In [ ]:
campaign_exposure = {'ES_BullishCollarBW_Short_EXO': 1.0,
'ES_CallSpread_Long_EXO': 1.0,
'ES_CallSpread_Long_SwingPoint_bullish-custom': 1.0,
'ES_CallSpread_Short_EXO': 1.0,
'ES_ContFut_Short_EXO': 1.0,
'ES_PutSpread_Long_BollingerBands_bearish-custom': 1.0,
'ES_PutSpread_Long_EXO': 1.0,
}

In [ ]:
campaign_dict = {}
campaign_stats = {'NetProfit': 0.0, 'TradesCount': 0, 'CommissionSum': 0.0}
campaign_deltas_dict = {}

for camp_name, exposure in campaign_exposure.items():
    if camp_name in swmdf:
        swarm_name = camp_name
        campaign_dict[swarm_name] = swmdf[swarm_name] * exposure
        
        # TODO: implement swarm statistics
        for s in swm_data:            
            if s['swarm_name'] != swarm_name:
                continue
            series = s['swarm_series']
            
            _delta_arr = campaign_deltas_dict.setdefault(s['instrument'], pd.Series(0, index=series['delta'].index))                                    
            campaign_deltas_dict[s['instrument']] = pd.concat([_delta_arr, series['delta']], axis=1).sum(axis=1)
    
campaign_portfolio = pd.DataFrame(campaign_dict).ffill()
campaign_equity = campaign_portfolio.sum(axis=1)
campaign_deltas = pd.DataFrame(campaign_deltas_dict)

### Campaign members equities

In [ ]:
figsize(10,10)
campaign_portfolio.plot();

### Campaign deltas by product

In [ ]:
campaign_deltas.plot();

### Cumulative campaign equity

In [ ]:
campaign_portfolio.sum(axis=1).plot()

# Stage 3: Campaign save

In [ ]:
from tradingcore.execution_manager import ExecutionManager
from tradingcore.campaign import Campaign
from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.assetindex_mongo import AssetIndexMongo

assetindex = AssetIndexMongo(MONGO_CONNSTR, MONGO_EXO_DB)
datasource = DataSourceMongo(MONGO_CONNSTR, MONGO_EXO_DB, assetindex, futures_limit=10, options_limit=10, exostorage=storage)
exmgr = ExecutionManager(MONGO_CONNSTR, datasource, dbname=MONGO_EXO_DB)

### Set campaign values

In [ ]:
CAMPAIGN_NAME = 'CL_BEAR_TEST'
CAMPAIGN_DESCRIPTION = 'Bearish Expression of Crude Oil'

### Run to save campaign to DB

#### Warning: old campaign data will be overwritten

In [ ]:
campaign_dict = {
    'name': CAMPAIGN_NAME,
    'description': CAMPAIGN_DESCRIPTION,
    'alphas': {}
}

for k,v in campaign_exposure.items():
    campaign_dict['alphas'][k] = {'qty': v,},

cmp = Campaign(campaign_dict, datasource)
exmgr.campaign_save(cmp)